## Анализ рынка продажи смартфонов за 01.11.2021 - 16.02.2022

Данный анализ проводился с целью выявления самых популярных моделей телефонов для заказа чехлов на них и торговли этими чехлами на маркетплейсе. 

Импортируем необходимые для работы библиотеки

In [1]:
import pandas as pd
import os # для работы с файлами

С сайта shopstat.ru забираем необходимую статистику. Статистику можно скачать в формате .csv, по 50 записей в одном файле.

В данном случае я вручную загрузил 20 файлов (1000 записей) в категории "топ продаж мобильных телефонов за 01/11/2021 - 16/02/2022". Данные взяты с маркетплейса **ozon.ru**

Создадим ДатаФрейм из первого файла, куда будем присоединять остальные 

In [2]:
full_df = pd.read_csv('1.csv', encoding='cp1251')

Циклом присоединяем остальные ДатаФреймы.

In [3]:
for i in range(1,20):
    i += 1
    n = str(i)+".csv"
    temp_df = pd.read_csv(n, encoding='cp1251')
    full_df = pd.concat([full_df, temp_df])

Проверим количество строк, колонок, удалим дубликаты, если есть.

In [4]:
full_df = full_df.drop_duplicates()

In [5]:
full_df.shape

(1000, 14)

Посмотрим содержимое, первые три строки:

In [6]:
full_df.head(3)

,SKU,Название,Категория,Бренд,Остаток,Отзывов,Минимальная цена,Максимальная цена,Средняя цена,Продаж,Выручка,Ссылка на товар,Фото,Рейтинг
0,251217817,"Смартфон Samsung Galaxy A02 2/32GB, синий",Смартфоны,Samsung,51,2,9221,9221,8857,1738,15367260,https://www.ozon.ru/context/detail/id/251217817/,https://cdn1.ozone.ru/s3/multimedia-8/60553520...,4.50
1,262327060,"Смартфон Xiaomi Redmi 9A 2/32GB, зеленый",Смартфоны,Xiaomi,25,4,8900,8900,8546,642,5707800,https://www.ozon.ru/context/detail/id/262327060/,https://cdn1.ozone.ru/s3/multimedia-1/60809683...,5.00
2,254513996,"Смартфон Poco X3 Pro 8/256GB, синий",Смартфоны,Poco,375,4700,24980,24980,23186,392,9246990,https://www.ozon.ru/context/detail/id/254513996/,https://cdn1.ozone.ru/s3/multimedia-f/60572614...,4.92


Очистим данные - уберем ненужные колонки, такие как - SKU, остаток, мин.цену, макс.цену, ссылки, фото и рейтинг

In [7]:
full_df.drop(['SKU', 'Категория', 'Остаток', 'Минимальная цена', 'Максимальная цена', 'Ссылка на товар', 'Фото', 'Рейтинг'], axis=1, inplace=True) 
full_df.head(3)

,Название,Бренд,Отзывов,Средняя цена,Продаж,Выручка
0,"Смартфон Samsung Galaxy A02 2/32GB, синий",Samsung,2,8857,1738,15367260
1,"Смартфон Xiaomi Redmi 9A 2/32GB, зеленый",Xiaomi,4,8546,642,5707800
2,"Смартфон Poco X3 Pro 8/256GB, синий",Poco,4700,23186,392,9246990


Теперь нам необходимо вычистить даннные - убрать "Смартфон" из названия, разделить производителя и модель устройства, убрать цвет и сгруппировать данные по моделям ("Смартфон HTC Desire 10 Pro, синий" для нас ничем не отличается от "Смартфон HTC Desire 10 Pro, черный")

In [8]:
def split_brand(brand_name):
    return brand_name.split(' ')[1]

In [9]:
full_df.insert(1, 'brand_name', full_df['Название'].apply(split_brand))

In [10]:
def model_name(model_name):
    x = len(model_name.split(' ')[2:8])
    if x < 5:
        return str(model_name.split(' ')[2]) + (' ') + str(model_name.split(' ')[3])
    else: 
        return str(model_name.split(' ')[2]) + (' ') + str(model_name.split(' ')[3]) + (' ') + str(model_name.split(' ')[4])


In [11]:
full_df.insert(2, 'model_name', full_df['Название'].apply(model_name))

Удаляем лишнее и переименовываем колонки на английский, для удобства пользования

In [12]:
full_df.drop(['Название', 'Бренд'], axis=1, inplace=True) 

In [13]:
# Также приведем все записи в датафрейме к нижнему регистру:
full_df.brand_name = full_df.brand_name.str.lower()
full_df.model_name = full_df.model_name.str.lower()

In [14]:
full_df = full_df.rename(columns = {'Средняя цена': 'mean_price', 'Продаж': 'total_sales', 'Выручка': 'total_money',  'Отзывов': 'comments'})

**Группируем все данные по производителю и модели устройства**

In [15]:
full_df.groupby(['brand_name', 'model_name'], as_index = False) \
    .agg({'total_sales': 'sum'}) \
    .sort_values('total_sales', ascending = False) \
    .query ('total_sales > 0')

,brand_name,model_name,total_sales
205,samsung,galaxy a12,15252
294,xiaomi,redmi 9a,11075
309,xiaomi,redmi note 10s,8037
261,vivo,y53s helio g80,4761
220,samsung,galaxy a32 4/64gb,3615
...,...,...,...
151,oukitel,"c25 4/32gb,",7
4,alcatel,"5033fr 1/32gb,",7
118,mini,"x50 4,0-дюймовый двухъядерный",7
191,rino4,"pro-11, черный",7


Смотрим внимательно на данные столбца model_name и видим, что в исходном ДФ часть данных была собрана с ошибками. 

Придется эти данные откорректировать вручную, иначе группировка по модели не имеет смысла ('Redmi 9A' и 'Redmi 9A32GB,' для нас это одно и то же, а в ДФ это разные строки с разной суммой по продажам)

In [16]:
full_df.groupby(['brand_name', 'model_name'], as_index = False) \
    .agg({'total_sales': 'sum'}) \
    .sort_values('total_sales', ascending = False) \
    .query ('total_sales > 0') \
    .model_name.unique()

array(['galaxy a12', 'redmi 9a', 'redmi note 10s', 'y53s helio g80',
       'galaxy a32 4/64gb', 'y31 4/64gb,', 'redmi 9c', 'galaxy а52',
       'galaxy a52 8/256gb', 'redmi 9a (китайская', 'y31 snapdragon 662',
       'galaxy a02', 'galaxy a 22', 'x3 pro', '8x восстановленный',
       'galaxy a12 (2021)', '11 lite 5g', '11t 8/256gb,', 'blade l210',
       'kf6m spark 7', 'redmi note 8', 'iphone 11', 'bv6300 3/32gb,',
       'pixel 4 xl', 'c25s 4/128gb,', 'spark 8p 4/64gb,', 'pixel 4',
       'pixel 3', '6 pro восстановленный', 'iphone 12', 'pixel 3a xl',
       'note 10s', 'c11 (2021)', '8x восстановленный телефон',
       'galaxy m32 6/128gb', 'pixel 3a', 'c25s 4/128gb water', 'redmi 10',
       'redmi note 9', 'blade l9', 'spark 7', 'a54 4/64gb,',
       'p20 lite 2018', 'v21e 8/128gb,', 'redmi note 10',
       'note 4x восстановленный', 'nova 3i восстановленный',
       '5046l 2/16gb,', '6631g surf', 'c21y 3/32gb,', 'p20 pro',
       'galaxy m12', '6051g 2/16gb,', 'wildfire e2',
  

**Самая долгая и нудная часть по вычищению данных:**

In [17]:
full_df.model_name.replace({
       'galaxy a32 4/64gb': 'galaxy a32', 'y31 4/64gb,': 'y31', 'galaxy a52 8/256gb': 'galaxy a52',         'redmi 9a (китайская': 'redmi 9a', 'y31 snapdragon 662': 'y31',       'galaxy a 22': 'galaxy a22', '8x восстановленный': '8x', '11 lite 5g': '11 lite', '11t 8/256gb,': '11t',        'bv6300 3/32gb,': 'bv6300', 'c25s 4/128gb,': 'c25s', 'spark 8p 4/64gb,': 'spark 8p',       '6 pro восстановленный': '6 pro', '8x восстановленный телефон': '8x',       'galaxy m32 6/128gb': 'galaxy m32', 'c25s 4/128gb water': 'c25s',        'a54 4/64gb,': 'a54', 'v21e 8/128gb,': 'v21e', 
       'note 4x восстановленный': 'note 4x', 'nova 3i восстановленный': 'nova 3i',       '5046l 2/16gb,': '5046l', 'c21y 3/32gb,': 'c21y',        '6051g 2/16gb,': '6051g',       'note4 восстановленный': 'note4', 'redmi 10 helio': 'redmi 10',         'c25s 4/64gb,': 'c25s',       'redmi 9c helio': 'redmi 9c', '5745l clever32gb,': '5745l',        'galaxy a32 4/128gb': 'galaxy a32',      'c11 (2021)32gb,': 'c11 (2021)',        '4030g 1/16gb,': '4030g', '6051g 2/32gb,': '6051g',       'redmi 9a 32gb': 'redmi 9a', '5031g fun night': '5031g',         'galaxy a12 3/32gb,': 'galaxy a12',       'sm-a032fzkdser 2/32gb,': 'sm-a032fzkdser', 'redmi 9a32gb,': 'redmi 9a',
       'galaxy a12 4/64gb,': 'galaxy a12', 'galaxy a03 core': 'galaxy a03',         'redmi 9c nfc': 'redmi 9c',       '9 восстановленный': '9', 'iphone 13 128gb,': 'iphone 13',        '7.2 6/128gb,': '7.2', 'galaxy a52256gb,':  'galaxy a52',       'galaxy m12 4/64gb': 'galaxy m12', 'bv4900 3/32gb,': 'bv4900',       'y31 4/64gb ocean': 'y31', 'c11 2/32gb,': 'c11', 'y5p 2/32gb,': 'y5p',       'galaxy m12 4/64gb,': 'galaxy m12',        'galaxy a03s32gb,': 'galaxy a03s', '1.4 3/64gb,': '1.4',         'galaxy a12 4/128gb,': 'galaxy a12',       'a25 1/16gb,': 'a25', 'blade a31 2/32gb': 'blade a31',       'galaxy a52 4/128gb': 'galaxy a52', 'redmi 9a 4gb': 'redmi 9a',
       'a74 4/128gb,': 'a74', '5016g choice': '5016g',         '10 3/32gb,': '10',       'g20 4/64gb,': 'g20',        'f3 6.67"fhd+/120hz/sd870/and11/48+8+5mp/20mp/nfc/4520mah': 'f3',       'galaxy a0232gb,': 'galaxy a02', 'galaxy a02 2/32gb': 'galaxy a02',       '5745l clever': '5745l', 'redmi 4x восстановленный': 'redmi 4x',         'p30 6/128gb,': 'p30',       '6022g 2/16gb,':'6022g', 'sm-a022 galaxy a02': 'galaxy a02',         'pixel-3 4/64gb,': 'pixel-3',       'pixel-3 4/128gb,': 'pixel-3', '8i 4/64gb,': '8i',       '1 1/1gb,': '1', '6022g16gb, фиолетовый': '6022g', 
    'redmi 9a ':'redmi 9a', 'x3 pro256gb,': 'x3 pro',       'blade l210 1/32gb': 'blade l210',       'note 7 восстановленный': 'note 7', 'galaxy a03s64gb,': 'galaxy a03',        '6630 3/32gb,': '6630', '7.2 4/,': '7.2', '5047l like': '5047l',       'y31 4/128gb,': 'y31', 'galaxy a52128gb,': 'galaxy a52',         'f3 8/256gb,': 'f3', 'c30 2/32gb,': 'c30',       'galaxy a12 3/32gb':'galaxy a12', 'c21y 4/64gb,': 'c21y',        'a48 2/32gb,': 'a48', 'a48 2/32gb gradation': 'a48',       '1 1/8gb,': '1', 'y5p black,': 'y5p',       'redmi 10 4/64gb': 'redmi 10', 'c20 2/32gb,': 'c20',       'pixel-2xl 4/128gb,': 'pixel-2xl', '8x 4/128gb,': '8x', 
       '5 2/16gb,': '5', '3/32gb, розовый': '3', '8128gb, черный': '8',       'pixel-3xl 4/128gb,': 'pixel-3xl',       'galaxy m22 4/128gb': 'galaxy m22', 'k7x, черный': 'k7x',     'x3 6/128gb,': 'x3',        '8 6/128gb,': '8',        'galaxy a52 sm-a525f': 'galaxy a52', 'y53s 6/128gb,': 'y53s',       '50 8/128gb,': '50', 'galaxy a22 4/128gb': 'galaxy a22', '11t 8/128gb,':'11t',
       'm3 4/128gb blue': 'm3', 'galaxy a12 sm-a127f': 'galaxy a12',       'a55 4/128gb,': 'a55', 'redmi 9c 4/128gb': 'redmi 9c',        'iphone 13 256gb,': 'iphone 13', '3l 5029y': '3l',       'c21 3/32gb,':'c21', 'y6p med-lx9n': 'med-lx9n',        'wp5 pro 4/64gb,':'wp5 pro', 'galaxy m32 sm-m325': 'galaxy m32',         '2 2021':'2',       '7 4/64gb,': '7',        '9a 4/64gb,': '9a', 'hot 11s nfc': 'hot 11s', 'c30 3/64gb,':'c30',         '5740 1/16gb,': '5740',       '5533g fresh cherry': '5533g', 'pixel-2 4/128gb,': 'pixel-2',         '6645l  element': '6645l',       'note 5a восстановленный': 'note 5a', '1/8gb, красный': '1',    
       'bv6600 4/64gb,': 'bv6600', '5 8/128gb,': '5', 'iphone 13 512gb,': 'iphone 13',      'x20 (ta-1341)': 'x20', 'pixel-3a 4/64gb,': 'pixel-3a',       '50 6/128gb,': '50', 'y31 4/64gb racing': 'y31', 'b2021 6/64gb,': 'b2021',        'a16 3/32gb,': 'a16',       'redmi 7a совершенно': 'redmi 7a', 'gt 8/128gb,': 'gt',       '20 4g (китайская': '20', 'pixel-3xl 4/64gb,': 'pixel-3xl',         'galaxy a22s 5g': 'galaxy a22s',       'blade l21032gb,': 'blade', 'g20 4/32gb, синий.': 'g20',       'c25s 4/64gb water': 'c25s', 'galaxy a12 (exynos)': 'galaxy a12',        'u11+ 4/128gb,': 'u11+',       'redmi 9c32gb,': 'redmi 9c',        'galaxy a12 4/128gb': 'galaxy a12', '5533g fresh night': '5533g',
       'galaxya12 4/128gb,': 'galaxy a12', '11, черный': '11', 'wp17 8/128gb,': 'wp17',       'galaxy a52 sm-a525': 'galaxy a52', '6051g soul 2/32gb,': '6051g',        'blade a51 2/64gb,': 'blade a51',       'redmi 9c64gb,': 'redmi 9c', 'iphone xs как': 'iphone xs',        'pixel 4a 6/128gb,': 'pixel 4a', 'pro, голубой': 'pro',       '8i 4/128gb': '8i', '8i 4/64gb': '8i', 'g50 4/128gb,': 'g50',       'x3 6/128gb, синий.': 'x3', '1b 5002h': '1b', 'blade l210 голубой': 'blade l210',       's42h+ 32/,': 's42h+', 'c21-y 3/32gb blue': 'c21y' ,       'pova 2 4/128gb': 'pova 2', '3.4 3/64gb,': '3.4', 'a55 4/64gb,': 'a55',       '8i 4/128gb,': '8i', 'wp5 4/32gb,': 'wp5',
       'galaxy m52 5g': 'galaxy m52', 'galaxy a22 4/64gb': 'galaxy a22',        'gt neo2 5g': 'gt neo2',       'bison128gb, черный,': 'bison',        '11, (product)red':'11', 'redmi 10 (nfc)':'redmi 10',         'c11 2021 2/32gb': 'c11',       '11 128gb,': '11', '11 64gb,': '11', 'c30 (ta-1359)': 'c30',        '50 lite 6/128gb': '50 lite',       'it2320, белый': 'it2320',        'g3502 4g': 'g3502', 'pixel 5a 5g': 'pixel 5a',        '5997132 4/64gb,': '5997132', 'redmi note8': 'redmi note 8',        'plus 5,72-дюймовый двухъядерный': 'plus', 'galaxy a03 4/64gb': 'galaxy a03',        'y53s 6/128gb': 'y53s', 'c25 4/32gb,': 'c25', '5033fr 1/32gb,': '5033fr',
       'x50 4,0-дюймовый двухъядерный': 'x50', 'pro-11, черный': 'pro 11',       'bv9900e 6/128gb,': 'bv9900e', 'galaxy a52': 'galaxy а52',     'sm-a032fzkdser': 'galaxy a32',     'galaxy a03s': 'galaxy a03', 'a03 core': 'galaxy a03', 'p20 pro,': 'p20 pro'
    }, inplace = True)

**С 333 наименований сократили до 229.**

In [18]:
full_df.groupby(['brand_name', 'model_name'], as_index = False) \
    .agg({'total_sales': 'sum'}) \
    .sort_values('total_sales', ascending = False) \
    .query ('total_sales > 0') \
    .model_name.nunique()

229

**Посмотрим топ-150 моделей по продажам и экспортируем в excel для дальнейшей отправки:**

In [19]:
top150_by_sales = full_df.groupby(['brand_name', 'model_name'], as_index = False) \
    .agg({'total_sales': 'sum', 'mean_price': 'mean'}) \
    .sort_values('total_sales', ascending = False) \
    .query ('total_sales > 100') \
    .head(150).reset_index() \
    .drop(['index'], axis=1) \
    .round(2)
top150_by_sales.to_excel("top150_by_sales.xlsx") 
top150_by_sales

,brand_name,model_name,total_sales,mean_price
0,samsung,galaxy a12,16395,12792.72
1,xiaomi,redmi 9a,14475,8367.68
2,xiaomi,redmi note 10s,8037,19134.91
3,vivo,y31,5516,14859.06
4,vivo,y53s helio g80,4761,20023.75
...,...,...,...,...
115,infinix,hot 10 lite.,113,7975.00
116,huawei,p smart 2021,107,17066.67
117,oppo,reno 5 lite,107,23856.00
118,nokia,c20,102,7419.00


При экспорте с сайта shopstat.ru нельзя выбрать экспорт продаж в зависимости от даты. 

Я просмотрел эти данные на самом сайте и увидел аномалии - продажи распределяются неравномерно, а скачкообразно (например, 15-17 января нет продаж, 18 - много, 19 - опять нет). Также странно, что на некоторых телефонах много продаж, но нет отзывов. Поэтому, для того что проверить гипотезы и посмотреть популярность модели, попробуем еще использовать **сортировку по количеству отзывов:**

In [20]:
top150_by_comments = full_df.groupby(['brand_name', 'model_name'], as_index = False) \
    .agg({'total_sales': 'sum', 'comments': 'sum', 'mean_price': 'mean'}) \
    .sort_values('comments', ascending = False) \
    .query ('total_sales > 100') \
    .query ('comments > 0').reset_index() \
    .drop(['index'], axis=1) \
    .round(2)
top150_by_comments.to_excel("top150_by_comments.xlsx")
top150_by_comments

,brand_name,model_name,total_sales,comments,mean_price
0,samsung,galaxy a12,16395,80602,12792.72
1,poco,x3 pro,1848,23368,21919.55
2,apple,iphone 11,1325,20495,52623.54
3,apple,iphone 12,855,17495,69781.77
4,xiaomi,redmi 9a,14475,11812,8367.68
...,...,...,...,...,...
115,blackview,bv6300,1290,2,12218.00
116,alcatel,1,291,2,5012.50
117,bq,6630,146,2,8439.50
118,bq,5047l,130,2,4337.00
